# Text classification using Tensorflow: Sentiment on Reviews
# Hyperparameter Tuning

This notebook trains a binary classifier for sentiment analysis to classify restaurant food reviews as *negative* or *positive*, based on the text of the review where a sentiment score was previously calculated. 


In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade -q wandb
import wandb
from wandb.keras import WandbCallback
import os
import random
import numpy as np
import warnings
import pandas as pd
import shutil
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
import matplotlib.pyplot as plt
import tqdm
warnings.filterwarnings('ignore')

     |████████████████████████████████| 1.8 MB 4.1 MB/s 
     |████████████████████████████████| 145 kB 56.8 MB/s 
     |████████████████████████████████| 181 kB 79.7 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


In [ ]:
print('CUDA and NVIDIA GPU Information')
!/usr/local/cuda/bin/nvcc --version
!nvidia-smi
print('\n')
print('TensorFlow version: {}'.format(tf.__version__))
print('Eager execution is: {}'.format(tf.executing_eagerly()))
print('Keras version: {}'.format(tf.keras.__version__))
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

CUDA and NVIDIA GPU Information
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Tue Jul  5 04:48:26 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|            

In [ ]:
# Reproducibility in TF 
def init_seeds(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    session_conf = tf.compat.v1.ConfigProto()
    session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1,
                                            inter_op_parallelism_threads=1)
    os.environ['TF_CUDNN_DETERMINISTIC'] ='True'
    os.environ['TF_DETERMINISTIC_OPS'] = 'True'
    tf.random.set_seed(seed)
    sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(),
                                config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)
    return sess

In [ ]:
init_seeds(seed=42)

In [ ]:
%cd /content/drive/MyDrive/Yelp_Reviews/Data/

/content/drive/MyDrive/Yelp_Reviews/Data


In [ ]:
# Read tokenized reviews with stars filtered to 1, 2 and 5
df = pd.read_parquet('YelpReviews_NLP_125stars_tokenized.parquet')
print('Number of rows and columns:', df.shape)
df.head()

Number of rows and columns: (1541486, 2)


,stars_reviews,cleanReview
index,,
2,5.0,"[wonderful, place, get, lunch, staff, friendly..."
3,5.0,"[dedicated, loving, memory, gary, feldman, gre..."
5,5.0,"[love, place, everything, menu, delicious, lik..."
7,5.0,"[truely, hidden, gem, place, greatest, buffalo..."
9,5.0,"[seriously, great, fresh, fantastic, food, spo..."


In [ ]:
# Remove comma from tokenize to make one string
df['cleanReview'] = df['cleanReview'].apply(lambda x: ','.join(map(str, x)))
df.loc[:,'cleanReview'] =  df['cleanReview'].str.replace(r',', ' ', regex=True)
df.head()

,stars_reviews,cleanReview
index,,
2,5.0,wonderful place get lunch staff friendly effic...
3,5.0,dedicated loving memory gary feldman greatest ...
5,5.0,love place everything menu delicious like tuna...
7,5.0,truely hidden gem place greatest buffalo chick...
9,5.0,seriously great fresh fantastic food spot pric...


In [ ]:
# Recode to binary
df['stars_reviews'].mask(df['stars_reviews'] == 1, 0, inplace=True)
df['stars_reviews'].mask(df['stars_reviews'] == 2, 0, inplace=True)
df['stars_reviews'].mask(df['stars_reviews'] == 5, 1, inplace=True)

print('Number of reviews per group:')
df[['stars_reviews']].value_counts() 

Number of reviews per group:


stars_reviews
0.0              770743
1.0              770743
dtype: int64

In [ ]:
# Sample the number of reviews to balance with sentiment
df1 = df[df.stars_reviews==0]
df1 = shuffle(df1)
df1 = df1.sample(n=414937)

In [ ]:
# Sample the number of reviews to balance with sentiment
df2 = df[df.stars_reviews==1]
df2 = shuffle(df2)
df2 = df2.sample(n=414937)

# Combine sampled groups 
df = pd.concat([df1, df2])
df = shuffle(df)

del df2

df.shape

(829874, 2)

In [ ]:
# Convert data types
df[['cleanReview']] = df[['cleanReview']].astype('str')
df['stars_reviews'] = df['stars_reviews'].astype('float32')

In [ ]:
# Set up features and label
features = df['cleanReview']
labels = df['stars_reviews']

In [ ]:
# Load pandas df to tf
df = tf.data.Dataset.from_tensor_slices((features, labels))

In [ ]:
# Define components for paritioning data
df_size = df.cardinality().numpy()
train_size = int(0.8 * df_size)
val_size = int(0.10 * df_size)
test_size = int(0.10 * df_size)
BATCH_SIZE = 32

In [ ]:
# Prepare train/test sets
df = df.shuffle(800_000, seed=42, reshuffle_each_iteration=False)
train = df.take(train_size).batch(BATCH_SIZE) 
test = df.skip(train_size)
val = test.skip(val_size).batch(BATCH_SIZE) 
test= test.take(test_size).batch(BATCH_SIZE) 

print('Train data split:', len(train))
print('Eval data split', len(val))
print('Test data split', len(test))

Train data split: 20747
Eval data split 2594
Test data split 2594


# Sentiment analysis




### Prepare the data for training

Next, the reviews are tokenized and vectorized using `tf.keras.layers.TextVectorization` layer. Tokenization refers to splitting strings into single words, or tokens. Vectorization converts the tokens into numbers as input for the neural network. Unique integer indices are created for each token by using `output_mode` = `int`. The `max_features` and `sequence_length` are defined for what is sampled.

In [ ]:
max_features = 50000
sequence_length = 300

vectorize_layer = layers.TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length)

The preprocessing layer is fit to the dataset by using `adapt` for the training data, which results in the model creating an index of strings to integers.

In [ ]:
# Make a review-only dataset and then call adapt
train_text = train.map(lambda x, y:x)
vectorize_layer.adapt(train_text)

Now, define a function to examine how the created layer preprocesses the data:



In [ ]:
def vectorize_text(cleanReview, stars_reviews):
  cleanReview = tf.expand_dims(cleanReview, -1)
  return vectorize_layer(cleanReview), stars_reviews

In [ ]:
# Sample a batch reviews and labels
text_batch, label_batch = next(iter(train))
first_review, first_label = text_batch[0], label_batch[0]
print('Review', first_review)
print('Label', first_label)
print('Vectorized review', vectorize_text(first_review, first_label))

Review tf.Tensor(b'food terribly salted waiter personality never filled husband drink never asked everything good till stacked plate end', shape=(), dtype=string)
Label tf.Tensor(0.0, shape=(), dtype=float32)
Vectorized review (<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[   2, 2137, 1472,  157, 2144,   26,  709,  209,   34,   26,   57,
          77,    4, 1475, 3873,  137,  257,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,   

In [ ]:
print('12870 ---> ', vectorize_layer.get_vocabulary()[12870])
print(' 3130 ---> ', vectorize_layer.get_vocabulary()[3130])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

12870 --->  refers
 3130 --->  aged
Vocabulary size: 50000


Then, apply the `TextVectorization` layer to the train, validation, and test sets.

In [ ]:
train_ds = train.map(vectorize_text)
val_ds = val.map(vectorize_text)
test_ds = test.map(vectorize_text)

Configure the dataset for performance by using `.cache()` and `.prefetch()`.




In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Hyperparameter Tuning

In [ ]:
# Set path for results
%cd /content/drive/MyDrive/Yelp_Reviews/Models/DL/BoW/ReviewStars/Models/

In [ ]:
# Log into wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Configure the sweep 
sweep_config = {
    'method': 'random',
    'metric': {
      'name': 'BinaryAccuracy',
      'goal': 'maximize'},
    'parameters': {
        'epochs': {
            'values': [3, 5, 7]},
        'batch_size': {
            'values': [8, 4, 2, 1]},
        'embedding_dim': {
            'values': [128, 64, 32]},        
        'dropout': {
            'values': [0.2, 0.3, 0.4, 0.5]},
        'weight_decay': {
            'values': [0.0005, 0.005, 0.05]},
        'learning_rate': {
            'values': [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01]},
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop']}}
            }

## 2. Initialize the Sweep

In [ ]:
# Initialize a new sweep
sweep_id = wandb.sweep(sweep_config, entity='aschultz', 
                       project='tf_textclass_reviewstars')

Create sweep with ID: ywp4qai8
Sweep URL: https://wandb.ai/aschultz/tf_textclass_reviewstars/sweeps/ywp4qai8


### Define Neural Network


In [ ]:
# The sweep calls this function with each set of hyperparameters
def train():
  
    # Default values for hyperparameters
    config_defaults = {
        'epochs': 5,
        'batch_size': 8,
        'weight_decay': 0.0005,
        'learning_rate': 0.01,
        'optimizer': 'adam',
        'embedding_dim': 32,
        'dropout': 0.5,
        'momentum': 0.9,
        'seed': 42}

    # Initialize a new wandb run
    wandb.init(config=config_defaults)
    
    # Config hyperparameters and inputs
    config = wandb.config
    
    # Define the model architecture 
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(max_features + 1, config.embedding_dim))
    model.add(tf.keras.layers.Dropout(config.dropout))
    model.add(tf.keras.layers.GlobalAveragePooling1D())
    model.add(tf.keras.layers.Dropout(config.dropout))
    model.add(tf.keras.layers.Dense(1))

    # Define the optimizer
    if config.optimizer=='sgd':
      optimizer = SGD(learning_rate=config.learning_rate, decay=1e-5, 
                      momentum=config.momentum, nesterov=True)
    elif config.optimizer=='rmsprop':
      optimizer = RMSprop(learning_rate=config.learning_rate, decay=1e-5)
    elif config.optimizer=='adam':
      optimizer = Adam(learning_rate=config.learning_rate, beta_1=0.9, 
                       beta_2=0.999, clipnorm=1.0)
    elif config.optimizer=='nadam':
      optimizer = Nadam(learning_rate=config.learning_rate, beta_1=0.9, 
                        beta_2=0.999, clipnorm=1.0)

    # Define loss and metrics
    model.compile(loss=losses.BinaryCrossentropy(from_logits=True), 
                  optimizer=optimizer, 
                  metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

    # Fit model
    model.fit(train_ds, batch_size=config.batch_size,
              epochs=config.epochs,
              validation_data=(val_ds),
              callbacks=[WandbCallback(data_type='text', 
                                       validation_data=(val_ds)), 
                                       EarlyStopping(patience=10, 
                                                     restore_best_weights=True)]) 

## 3. Run the sweep agent

In [ ]:
# Run sweep for HPO
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: jfj72ec9 with config:
wandb: 	batch_size: 8
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 128
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0005
wandb: 	optimizer: adam
wandb: 	weight_decay: 0.005
wandb: Currently logged in as: aschultz. Use `wandb login --relogin` to force relogin


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING The data_type argument of wandb.keras.WandbCallback is deprecated and will be removed in a future release. Please use input_type instead.
wandb: WARNING Setting input_type = data_type.


Epoch 1/7
20747/20747 [==============================] - 372s 18ms/step - loss: 0.1875 - binary_accuracy: 0.9369 - val_loss: 0.1169 - val_binary_accuracy: 0.9617 - _timestamp: 1656996993.0000 - _runtime: 375.0000
Epoch 2/7
20747/20747 [==============================] - 354s 17ms/step - loss: 0.1096 - binary_accuracy: 0.9632 - val_loss: 0.1096 - val_binary_accuracy: 0.9643 - _timestamp: 1656997346.0000 - _runtime: 728.0000
Epoch 3/7
20747/20747 [==============================] - 354s 17ms/step - loss: 0.1026 - binary_accuracy: 0.9657 - val_loss: 0.1078 - val_binary_accuracy: 0.9646 - _timestamp: 1656997700.0000 - _runtime: 1082.0000
Epoch 4/7
20747/20747 [==============================] - 352s 17ms/step - loss: 0.0990 - binary_accuracy: 0.9670 - val_loss: 0.1070 - val_binary_accuracy: 0.9652 - _timestamp: 1656998053.0000 - _runtime: 1435.0000
Epoch 5/7
20747/20747 [==============================] - 350s 17ms/step - loss: 0.0966 - binary_accuracy: 0.9678 - val_loss: 0.1077 - val_binary_a

binary_accuracy,▁▇▇████
epoch,▁▂▃▅▆▇█
loss,█▂▂▁▁▁▁
val_binary_accuracy,▁▆▇█▇▇█
val_loss,█▃▂▁▁▂▁
best_epoch,3
best_val_loss,0.10704
binary_accuracy,0.96891
epoch,6
loss,0.09356
val_binary_accuracy,0.96507


wandb: Agent Starting Run: 2fk35uoq with config:
wandb: 	batch_size: 2
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 64
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0001
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.005


...

wandb: Agent Starting Run: au83gr0w with config:
wandb: 	batch_size: 2
wandb: 	dropout: 0.4
wandb: 	embedding_dim: 32
wandb: 	epochs: 3
wandb: 	learning_rate: 0.0005
wandb: 	optimizer: nadam
wandb: 	weight_decay: 0.0005


Epoch 1/3
20747/20747 [==============================] - 188s 9ms/step - loss: 0.2754 - binary_accuracy: 0.9048 - val_loss: 0.1551 - val_binary_accuracy: 0.9473 - _timestamp: 1657067317.0000 - _runtime: 194.0000
Epoch 2/3
20747/20747 [==============================] - 188s 9ms/step - loss: 0.1462 - binary_accuracy: 0.9504 - val_loss: 0.1330 - val_binary_accuracy: 0.9551 - _timestamp: 1657067504.0000 - _runtime: 381.0000
Epoch 3/3
20747/20747 [==============================] - 188s 9ms/step - loss: 0.1320 - binary_accuracy: 0.9554 - val_loss: 0.1248 - val_binary_accuracy: 0.9580 - _timestamp: 1657067691.0000 - _runtime: 568.0000


binary_accuracy,▁▇█
epoch,▁▅█
loss,█▂▁
val_binary_accuracy,▁▆█
val_loss,█▃▁
best_epoch,2
best_val_loss,0.12478
binary_accuracy,0.95536
epoch,2
loss,0.13204
val_binary_accuracy,0.95798


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: bqlsm0i1 with config:
wandb: 	batch_size: 2
wandb: 	dropout: 0.4
wandb: 	embedding_dim: 128
wandb: 	epochs: 7
wandb: 	learning_rate: 0.005
wandb: 	optimizer: sgd
wandb: 	weight_decay: 0.0005


Epoch 1/7
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6790 - binary_accuracy: 0.5710 - val_loss: 0.6732 - val_binary_accuracy: 0.5868 - _timestamp: 1657067864.0000 - _runtime: 151.0000
Epoch 2/7
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6721 - binary_accuracy: 0.5877 - val_loss: 0.6644 - val_binary_accuracy: 0.6083 - _timestamp: 1657068066.0000 - _runtime: 353.0000
Epoch 3/7
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6581 - binary_accuracy: 0.6186 - val_loss: 0.6425 - val_binary_accuracy: 0.6776 - _timestamp: 1657068212.0000 - _runtime: 499.0000
Epoch 4/7
20747/20747 [==============================] - 146s 7ms/step - loss: 0.6265 - binary_accuracy: 0.6776 - val_loss: 0.6000 - val_binary_accuracy: 0.7697 - _timestamp: 1657068358.0000 - _runtime: 645.0000
Epoch 5/7
19892/20747 [===========================>..] - ETA: 5s - loss: 0.5771 - binary_accuracy: 0.7462